In [3]:
import numpy as np
import pandas as pd
import librosa as lr
import matplotlib.pyplot as plt
from types import SimpleNamespace

### **Audio Configuration**

In [5]:
audio_config = SimpleNamespace(
    base_path = "/workspaces/codespaces-blank/assets/audio/",
    sr = 44100,
    mono = True,
)

### **1. Read Audio.**

In [6]:
def get_audio_contents(path):
    audio_content, _ = lr.load(audio_config.base_path + path)
    audio_length = len(audio_content) // audio_config.sr
    sec_by_sec = []

    for sec in range(audio_length):
        segment_start_idx = audio_config.sr * sec
        segment_end_idx = audio_config.sr * (sec + 1)
        sec_by_sec.append(audio_content[segment_start_idx: segment_end_idx])
    
    return sec_by_sec

### **2. Make Spectrogram**

In [7]:
def make_spectrogram(audio_contents):
    sec_by_sec_spectrogram = []
    for aud_cnt in audio_contents:
        spectrogram_of_ith_sec = lr.stft(aud_cnt)
        sec_by_sec_spectrogram.append(spectrogram_of_ith_sec)

    return sec_by_sec_spectrogram

## **Pipeline**

In [9]:
class FingerprintPipeline:
    def __init__(self, reader_function, spectrogram_genrator_function):
        self.audio_configs = audio_config
        self.audio_content_reader = reader_function
        self.spectrogram_generator = spectrogram_genrator_function
        self.__audio_fingerprint = None

    def __fingerprint__(self, audio_path):
        audio_contents_sec_by_sec = self.audio_content_reader(audio_path)
        spectrogram_matrix = self.spectrogram_generator(audio_contents_sec_by_sec)
    
    def get_fingerprint(self):
        return self.__audio_fingerprint

In [10]:
fingerprint_pipeline = FingerprintPipeline(get_audio_contents, make_spectrogram)